In [1]:
# !ml GCC
# !ml Java
# !ml glibc

In [11]:
from pathlib import Path
from aicsimageio import AICSImage
import dask
from dask.distributed import LocalCluster, Client
import dask.array as da
import time
from skimage.io import imread

# import pyclesperanto_prototype as cle
import napari
import matplotlib.pyplot as plt
%matplotlib inline

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
# !pip install numpy --upgrade
import numpy as np

In [3]:
from dask.distributed import Client, Scheduler, Worker, Nanny, SpecCluster
import multiprocessing
import psutil

In [4]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 11
Total threads: 22,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37967,Workers: 11
Dashboard: http://127.0.0.1:8787/status,Total threads: 22
Started: 1 minute ago,Total memory: 200.00 GiB
Comm: tcp://127.0.0.1:35432,Total threads: 2
Dashboard: http://127.0.0.1:37631/status,Memory: 18.18 GiB
Nanny: tcp://127.0.0.1:34136,


Trying to debug why the file will not be read properly - AICSImage is used to quickly read the file (without loading it all into RAM) and therefore extract information such as `dtype` and `shape` to create the dask stack -> this produces an error :
shape must be int, list or tuple not NoneType
Manually put in `dtype` and `shape` instead of automating this 

In [20]:
%%time
folder_path = Path('Data')
file_pattern = '20191104_ABAT029_REG1-CTRL_DAPI_CD31-ENDOM-680_Th-647_hCD45-33-488_aSMA-CY3_aGFP-594.ome.btf'
filenames = []

def read_file(file_path):
    image = imread(file_path)
    return image

for file in folder_path.glob(file_pattern):
    filenames = np.append(filenames, file)
filenames = sorted(filenames)

# reads sample file to obtain file shape
print('Reading first file')
read_time = time.time()
sample = AICSImage(filenames[0])
print(f'Reading first file took {time.time() - read_time} s'
      f'')
# create dask stack of lazy image readers
print('Creating lazy image readers')
lazy_time = time.time()
lazy_reader = dask.delayed(read_file)  # lazy reader
lazy_arrays = [lazy_reader(fn) for fn in filenames]
print(f'Creating lazy image readers took {time.time() - lazy_time} s')

print('Creating dask stack')
stack_time = time.time()
dask_arrays = [
    da.from_delayed(delayed_reader, shape=(6, 44, 28068, 5265), dtype=np.uint16) for delayed_reader in lazy_arrays]

# stack into one large dask.array
timelapse = da.stack(dask_arrays, axis=0)

Reading first file
Reading first file took 1.3361718654632568 s
Creating lazy image readers
Creating lazy image readers took 0.0006017684936523438 s
Creating dask stack
CPU times: user 21.4 ms, sys: 18.5 ms, total: 39.9 ms
Wall time: 1.35 s


In [21]:
%%time
timelapse = da.squeeze(timelapse)

CPU times: user 596 µs, sys: 0 ns, total: 596 µs
Wall time: 604 µs


In [22]:
%%time
timelapse

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


dask.array<getitem, shape=(6, 44, 28068, 5265), dtype=uint16, chunksize=(6, 44, 28068, 5265), chunktype=numpy.ndarray>

In [23]:
# %%time
# timelapse.compute()

In [26]:
%%time
rechunked = da.rechunk(timelapse, (1, 44, 28068, 5265))
rechunked

CPU times: user 1.61 ms, sys: 126 µs, total: 1.74 ms
Wall time: 6.22 ms


dask.array<rechunk-merge, shape=(6, 44, 28068, 5265), dtype=uint16, chunksize=(1, 44, 28068, 5265), chunktype=numpy.ndarray>

In [ ]:
%%time
rechunked.compute()

### (OLD) Attempting to process things

In [16]:
def process_image(image):
    import time

    start_time = time.time()
    
    # push image to GPU memory and show it
    gpu_input = cle.push(image)
    # print(gpu_input)
    
    # gaussian blur
    sigma = 2.0
    gpu_blurred = cle.gaussian_blur(gpu_input, sigma_x=sigma, sigma_y=sigma, sigma_z=sigma)

    # detect maxima
    gpu_detected_maxima = cle.detect_maxima_box(gpu_blurred)
    
    # threshold
    threshold = 300.0
    gpu_thresholded = cle.greater_constant(gpu_blurred, constant=threshold)

    # mask
    gpu_masked_spots = cle.mask(gpu_detected_maxima, gpu_thresholded)

    # label spots
    gpu_labelled_spots = cle.connected_components_labeling_box(gpu_masked_spots)
    # show_labels(gpu_labelled_spots)
    
    number_of_spots = int(cle.maximum_of_all_pixels(gpu_labelled_spots))
    # print("Number of detected spots: " + str(number_of_spots))
    
    # label map closing
    number_of_dilations = 10
    flip = cle.create_labels_like(gpu_labelled_spots)
    flop = cle.create_labels_like(gpu_labelled_spots)
    flag = cle.create([1,1,1])
    cle.copy(gpu_labelled_spots, flip)

    for i in range (0, number_of_dilations) :
        cle.onlyzero_overwrite_maximum_box(flip, flag, flop)
        cle.onlyzero_overwrite_maximum_diamond(flop, flag, flip)

    # erode labels
    flap = cle.greater_constant(flip, constant=1)
    number_of_erosions = 4
    for i in range(0, number_of_erosions):
        cle.erode_box(flap, flop)
        cle.erode_box(flop, flap)

    gpu_labels = cle.mask(flip, flap)
    
    # get result back from GPU as numpy array
    result = cle.pull(gpu_labels).astype(np.uint16)
        
    print("Processing took " + str(time.time() - start_time) + " s")

    return result

In [22]:
def simple(image):
    import time
    start_time = time.time()
    # gpu_input = cle.push(image)
    
    radius = 10
    background_subtracted = cle.top_hat_box(image, radius_x=radius, radius_y=radius, radius_z=radius)

    labels = cle.voronoi_otsu_labeling(background_subtracted, spot_sigma=2, outline_sigma=1)

    # result = cle.pull(labels).astype(np.uint16)

    print("Processing took " + str(time.time() - start_time) + " s")
    return labels

#### Using `da.map_overlap`

In [23]:
%%time
overlapped_labels = da.map_overlap(simple, rechunked, dtype=np.uint16)

Processing took 0.11984515190124512 s
CPU times: user 242 ms, sys: 37.5 ms, total: 279 ms
Wall time: 626 ms


In [24]:
print(f'Number of tasks is for 64 chunks: {len(overlapped_labels.dask)}')

Number of tasks is for 64 chunks: 379


In [25]:
overlapped_labels.visualize(filename='overlapped_labels.svg', optimize_graph=True)

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

In [27]:
%%time
overlapped_labels.compute()

Exception: LogicError(<pyopencl._cl._ErrorRecord object at 0x7f431aba6530>)

#### Trying just dasl_image gaussianblur

In [15]:
%%time
from dask_image import ndfilters
filtered = ndfilters.gaussian_filter(rechunked, sigma=5, mode='constant')

CPU times: user 25.3 ms, sys: 7.85 ms, total: 33.1 ms
Wall time: 711 ms


In [16]:
print(f'Number of tasks is for 64 chunks: {len(filtered.dask)}')

Number of tasks is for 64 chunks: 3228


In [47]:
filtered.visualize(filename='gaussian_blur.svg', optimize_graph=True)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

### (old) Trying to process data

In [50]:
def segmentation(image):
    background_removed = cle.top_hat_box(image, radius_x=10, radius_y=10, radius_z=10)
    segmented = cle.voronoi_otsu_labelling(backgroung_removed, spot_sigma=2, outline_sigma=1)
    
    segmented_host = da.from_array(cle.pull(segmented))
    
    # memory clean up
    background_removed.data.release()
    segmented.data.release()
    
    return segmented_host

In [53]:
# create dask stack of lazy image readers
lazy_process_image = dask.delayed(segmentation)  # lazy reader
lazy_arrays = [lazy_process_image([n]) for n in range(0, rechunked.shape[0])]
dask_arrays = [
    da.from_delayed(lazy_array, shape=rechunked[0].shape, dtype=np.uint16)
    for lazy_array in lazy_arrays
]
# Stack into one large dask.array
dask_stack = da.stack(dask_arrays, axis=0)
# dask_stack = dask_stack.rechunk((1, 1, 603, 1272))
dask_stack

dask.array<stack, shape=(991, 1871, 965), dtype=uint16, chunksize=(1, 1871, 965), chunktype=numpy.ndarray>

In [13]:
print(timelapse.shape[0])

1


The above follows Robert Haase's example notebook on dask and napari interoperatability for the Drosophilia dataset, however, does not work on the Tribolium dataset due to a `MEM_ALLOCATION_ERROR` because chunking the timelapse by timepoints is still too much to handle.

Will attempt to chunk into smaller volumes within each timepoint, use `dask.map_blocks`, configure a dask cluster and schedule workers.

In [14]:
# # trying to gather device info
# cpu_count = multiprocessing.cpu_count()
# memory_count = psutil.virtual_memory().total
# print("CPU count:", cpu_count)
# print("System memory:", memory_count)

In [15]:
# specs = {
#     "gpu":{
#         "scale":4,
#         "resources":{
#             "GPU":1
#         }
#     }
# }

# worker_count = 0
# for v in specs.values():
#     worker_count += v["scale"]

# nthreads = cpu_count//worker_count
# memory_limit = int(memory_count*0.9)//worker_count # set to use 90% of the system memory to avoid crashing

# print("number of workers:", worker_count)
# print("threads per worker:", nthreads)
# print("memory limit per worker:", round(memory_limit/(1024*1024*1024),2), "GB")

In [16]:
# workers = {}

# for k, v in specs.items():
#     for i in range(v["scale"]):
#         workers["{}-{}".format(k,i)] = {
#             "cls":Nanny,
#             "options":{
#                 "nthreads": nthreads,
#                 "resources":v["resources"],
#                 "memory_limit": memory_limit
#                 }
#                 }

In [17]:
# scheduler = {'cls': Scheduler, 'options': {"dashboard_address": ':8787'}}
# cluster = SpecCluster(scheduler=scheduler, workers=workers)
# client = Client(cluster)
# client

In [18]:
def cle_gaussian_blur(image):
    blurred_device = cle.gaussian_blur(image)
    blurred_host = array.from_array(cle.pull(blurred_device))

    # memory clean up
    blurred_device.data.release()

    # variable with automatically removed at the end of function scope
    return blurred_host

In [19]:
# futures = []
# for i in range(4):
#     future = dask.delayed(cle_gaussian_blur)(timelapse)
#     futures.append(future)

In [20]:
# futures[0].visualize(filename='label_making.svg')

In [21]:
# # specify the worker for the compute process
# with dask.annotate(resources={'GPU': 1}):
#     res = dask.compute(*futures)

In [22]:
# res[0]

#### Rechunking


Following the `skan` 3D skeletal analysis

In [33]:
factor = np.array([4,4,4])  # even numbers
desired_chunksize = np.array(timelapse.shape[-3:])//factor

print(desired_chunksize)

[247 467 241]


In [34]:
rechunked = da.rechunk(timelapse, (1, desired_chunksize[0], desired_chunksize[1], desired_chunksize[2]))
rechunked

dask.array<rechunk-merge, shape=(1, 991, 1871, 965), dtype=uint16, chunksize=(1, 247, 467, 241), chunktype=numpy.ndarray>

In [35]:
processed = da.map_overlap(process_image, rechunked, dtype=np.uint16)
processed = da.squeeze(processed)
processed

dask.array<getitem, shape=(991, 1871, 965), dtype=uint16, chunksize=(247, 467, 241), chunktype=numpy.ndarray>

In [4]:
viewer = napari.Viewer()
viewer.add_image(timelapse, multiscale=False, scale=(1,1,1))
viewer.add_labels(overlapped_labels, multiscale=False, scale=(1,1,1))
napari.run()

Traceback (most recent call last):
  File "/camp/home/daov1/.conda/envs/pyclesperanto_env/lib/python3.9/site-packages/napari/_qt/qt_viewer.py", line 883, in _open_files_dialog
    self._qt_open(
  File "/camp/home/daov1/.conda/envs/pyclesperanto_env/lib/python3.9/site-packages/napari/_qt/qt_viewer.py", line 953, in _qt_open
    self.viewer.open(
  File "/camp/home/daov1/.conda/envs/pyclesperanto_env/lib/python3.9/site-packages/napari/components/viewer_model.py", line 1102, in open
    layers = self._open_or_raise_error(
  File "/camp/home/daov1/.conda/envs/pyclesperanto_env/lib/python3.9/site-packages/napari/components/viewer_model.py", line 1170, in _open_or_raise_error
    raise NoAvailableReaderError(
napari.errors.reader_errors.NoAvailableReaderError: No plugin found capable of reading /camp/home/daov1/daov/Example_Data/Example_4/20191104_ABAT029_REG1-CTRL_DAPI_CD31-ENDOM-680_Th-647_hCD45-33-488_aSMA-CY3_aGFP-594.ome.btf.


#### Using dask_image.ndmeasure.label 